In [ ]:
# lif sparsity test 

for i in np.arange(0.1, 3.0, 0.1):
     torch.manual_seed(42)

     # ONE CONV, ONE FC

     TIME = 8
     BATCH = 4
     CHANNEL_PIXEL = 3
     IMAGE_SIZE = 32

     CONV_OUT_CHANNEL = 64
     CONV_KERNEL_SIZE = 3


     LIF_V_THRESHOLD = i

     FC_OUT_FEATURES = 10




     # Create an instance of SYNAPSE_CONV
     synapse_conv = SYNAPSE_CONV(in_channels=CHANNEL_PIXEL, out_channels=CONV_OUT_CHANNEL, 
                                   kernel_size=CONV_KERNEL_SIZE, stride=1, padding=1, trace_const1=1, trace_const2=0.7)

     # Create an instance of LIF_layer
     lif_layer = LIF_layer(v_init=0.0, v_decay=0.8, v_threshold=LIF_V_THRESHOLD, v_reset=0.0, sg_width=1)

     # Create an instance of synapse_FC
     synapse_FC = SYNAPSE_FC(in_features=CONV_OUT_CHANNEL * IMAGE_SIZE * IMAGE_SIZE, 
                              out_features=FC_OUT_FEATURES, trace_const1=1, trace_const2=0.7)




     # Generate random spike input
     spike_input = torch.randn(TIME, BATCH, CHANNEL_PIXEL, IMAGE_SIZE, IMAGE_SIZE)



     # CONV
     conv_output_current = synapse_conv(spike_input)
     # print(conv_output_current.shape)
     # print(conv_output_current)


     # LIF
     lif_post_spike = lif_layer(conv_output_current)
     # print(lif_post_spike.shape)
     # print(lif_post_spike)

     # Calculate the ratio of 1's in lif_post_spike
     ones_ratio = (lif_post_spike == 1).sum() / lif_post_spike.numel()
     print("Ratio of 1's in lif_post_spike:", ones_ratio)



     # FC
     lif_post_spike_reshaped = lif_post_spike.view(TIME, BATCH, -1)
     fc_output_current = synapse_FC(lif_post_spike_reshaped)
     # print(fc_output_current.shape)
     # print(fc_output_current)